# Introdução 
Kannada é um língua falada predominantemente pelo povo de Karnataka, localizada na região sudoeste da Índia. Essa língua possui aproximandente 45 milhões de praticantes.

O objetivo desse projeto é construir uma Rede Neural Convulucional (CNN) que consiga interpretar a escrita kannadense a partir de um banco de dados, onde há mais de 60.000 imagens de 0 a 9, escritos a mão, em kannadense


In [ ]:
# Importando bibliotecas 
import pandas as pd
import seaborn as sns
import numpy as  np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

pd.options.display.max_columns = None
%matplotlib inline

In [ ]:
# Transformando datasets em dataframes
train = pd.read_csv('Kannada-MNIST/train.csv')
test = pd.read_csv('Kannada-MNIST/test.csv')
dig = pd.read_csv('Kannada-MNIST/Dig-MNIST.csv') 

___
# Conhecendo os dados 

Cada imagem possui 28 pixels em altura e 28 em comprimento, o que totaliza 784 pixels por imagem (28x28). Cada pixel tem o seu próprio valor associado, o que indica o tom claro e escuro do respectivo pixel.

Para podermos observar melhor essas imagens e como elas se comportam, realizaremos uma análise desses dados. 

In [ ]:
# Checando o tamanho do nosso dataframe 
print('O dataframe de TREINO possui {} LINHAS e {} COLUNAS'.format(train.shape[0], train.shape[1]))
print('O dataframe de TESTE possui {} LINHAS e {} COLUNAS'.format(test.shape[0], test.shape[1]))

In [ ]:
train.head()

In [ ]:
sns.barplot(train.label.value_counts().index, train.label.value_counts())
plt.xlabel('Labels')
plt.ylabel('Número de imagens')
plt.show()

Aqui plotamos um gráfico de barra para entendermos como está a distribuição das imagens. No nosso dataset, podemos observar que o número de imagens está uniformemente distribuido, o que nos leva a não precisar realizar algum tratamento ou manipulação em relação à quantidade para cada label.

In [ ]:
train.label.value_counts()

Para cada label, temos 6.000 imagens, o que é um bom número para se trabalhar. 

Em seguida, vamos plotar uma das imagens. Como os pixels estão distribuídos em 784 colunas, precisamos reagrupá-las de forma que estajam no formato 28x28. Por isso utilizaremos o reshape.

In [ ]:
num = 8
number = train.iloc[num,1:].values.reshape(28,28)
plt.imshow(number, cmap=plt.get_cmap('gray'))
plt.title('Imagem do número {} em Kannada'.format(str(num)))
plt.show()

Temos uma boa noção sobre o dataset que estamos trabalhando! Agora iniciaremos o processo de montagem do nosso CNN.

___
# Pré-processamento dos dados 

Para podermos utlizar as imagens como inputs no CNN, precisamos realizar alguns processamentos.

Iniciaremos com uma separação do nosso target 

In [ ]:
# Separando feature e targets 
X_train=train.drop('label',axis=1)
Y_train=train.label

Para podermos acelerar o processo de treinamento do nosso modelo, realizaremos uma normalização. Em outras palavras, já que os valores do pixel variam entre 0 e 255, dividiremos todos os valores por 255, desse modo, os inputs terão valores "leves" para serem trabalhadas no CNN. No final, os valores do pixels vão variar entre 0 e 1.

In [ ]:
X_train = X_train/255
test = test/255

Também, passaremos as variáveis dos labels de numéricas para categóricas: 

In [ ]:
from tensorflow.keras.utils import to_categorical

Y_train = to_categorical(Y_train)

Estamos prestes a montar o nosso CNN. O passo final antes de iniciar a montagem seria separar o nosso train com o auxilio do sklearn.

Decidimos utilizar a proporção de 70% e 30%

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size = 0.2, random_state=42) 

E mais um detalhe imporante!

Como dito antes, os pixels estão distribuídas de forma linear em 784 colunas, então precisamos agrupá-los no formato 28x28x1:

In [ ]:
x_train = x_train.values.reshape(-1,28,28,1) 
x_val = x_val.values.reshape(-1,28,28,1)
test = test.iloc[:, 1:].values.reshape(-1,28,28,1)

___
# Modelo 

Aqui se encontra uma breve explicação sobre CNN e algumas definições importantes: https://medium.com/data-hackers/uma-introdu%C3%A7%C3%A3o-as-redes-neurais-convolucionais-utilizando-o-keras-41ee8dcc033e

Agora iniciaremos a montagem do modelo!

In [ ]:
# Importantando bibliotecas 
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,MaxPool2D,Dropout,BatchNormalization, MaxPooling2D

In [ ]:
# Modelo inicial
model_1 = Sequential()
model_1.add(Conv2D(filters = 32,
                 kernel_size = (3,3),
                 padding = 'Same',
                 activation = 'relu',
                 input_shape = (28,28,1)))

model_1.add(MaxPool2D(pool_size = (2,2)))

model_1.add(Dropout(0.25))

model_1.add(Conv2D(filters = 64,
                 kernel_size = (3,3),
                 padding = 'Same',
                 activation = 'relu'))

model_1.add(MaxPool2D(pool_size = (2,2)))

model_1.add(Dropout(0.25))

model_1.add(Conv2D(filters = 128,
                 kernel_size = (3,3),
                 padding = 'Same',
                 activation = 'relu'))

model_1.add(Flatten())

model_1.add(Dense(256, activation = "relu"))

model_1.add(Dropout(0.5))

model_1.add(Dense(10, activation = "softmax"))

In [ ]:
model_1.summary()

### Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_train)

### Otimizadores 

In [ ]:
model_1.compile(optimizer = 'adam', loss='categorical_crossentropy' , metrics=['accuracy'])

### Callbacks

In [ ]:
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=300, restore_best_weights=True)

### Avaliando primeira performance 

In [ ]:
epochs = 20
batch_size = 1024

In [ ]:
history = model_1.fit(datagen.flow(x_train, y_train, batch_size = batch_size,), 
                      epochs = epochs,
                      validation_data = (x_val, y_val), 
                      validation_steps = 50,
                      steps_per_epoch = x_train.shape[0] // batch_size,
                      callbacks = [learning_rate_reduction, es])

# Resultado da primeira iteração
fig,ax=plt.subplots(2,1)
fig.set
x=range(1,1+epochs)
ax[0].plot(x,history.history['loss'],color='red')
ax[0].plot(x,history.history['val_loss'],color='blue')

ax[1].plot(x,history.history['accuracy'],color='red')
ax[1].plot(x,history.history['val_accuracy'],color='blue')
ax[0].legend(['trainng loss','validation loss'])
ax[1].legend(['trainng acc','validation acc'])
plt.xlabel('Number of epochs')
plt.ylabel('accuracy')

In [ ]:
model_1.evaluate(x_val, y_val, verbose=2);

### Matriz de Confusão 

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
y_predicted = model_1.predict(x_val)
y_grand_truth = y_val

y_predicted = np.argmax(y_predicted,axis=1)
y_grand_truth = np.argmax(y_grand_truth,axis=1)

cm = confusion_matrix(y_grand_truth, y_predicted)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(cm,fmt=".0f", annot=True, linewidths=0.1)
plt.xlabel("Predicted")
plt.ylabel("Grand Truth")
plt.show()

### Validação em cima de dados extras

In [ ]:
x_extra = train.drop('label',axis=1)
y_extra = train.label

x_extra = x_extra/255

x_extra = x_extra.values.reshape(-1,28,28,1) 
y_extra = to_categorical(y_extra)

In [ ]:
y_predicted_extra = model_1.predict(x_extra)
y_grand_truth_extra = y_extra

y_predicted_extra = np.argmax(y_predicted_extra, axis=1)
y_grand_truth_extra = np.argmax(y_grand_truth,axis=1)

cm_extra = confusion_matrix(y_grand_truth_extra, y_predicted_extra)

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(cm_extra,fmt=".0f", annot=True, linewidths=0.1)
plt.xlabel("Predicted")
plt.ylabel("Grand Truth")
plt.show()